# <center> Explore and cluster the neighborhoods in Toronto</center>

## Part1

a) Code to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes

In [1]:
# Import libraries need it for Part1
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np # library to handle data in a vectorized manner
print('Libraries Panda and Numpy imported.')

Libraries Panda and Numpy imported.


In [5]:
# Just using Panda
GetTable = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M', header=0)
df_pc = GetTable[0]
# Fixing the misspelling - Wikipedia!, really?
df_pc.rename(columns={'Neighbourhood':'Neighborhood'}, inplace=True)
df_pc.head()

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [6]:
df_pc.shape

(288, 3)

In [2]:
# Using the BeautifulSoup package 
# Web scraping packages
from bs4 import BeautifulSoup
import requests
print('BeatifulSoup and Libraries Requests imported.')

BeatifulSoup and Libraries Requests imported.


In [12]:
# Load URL that need in the dataframe
Table_url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
res = requests.get(Table_url)
soup = BeautifulSoup(res.text, 'html.parser')
table = soup.find('table',{'class':'wikitable sortable'}).tbody
table_rows=table.find_all('tr')

pc_list = []
for tr in table_rows:
    td = tr.find_all('td')
    row = [tr.text.strip() for tr in td if tr.text.strip()]
    if row:
        pc_list.append(row)

df_pcs = pd.DataFrame(pc_list, columns=["PostalCode", "Borough", "Neighborhood"])
print("Shape of Dataframe is - ",df_pcs.shape)
df_pcs.head()


Shape of Dataframe is -  (288, 3)


,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


b) Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [13]:
df_pcs = df_pcs[df_pcs.Borough!="Not assigned"]
print("Shape of Dataframe is - ",df_pcs.shape)
df_pcs.head(11)

Shape of Dataframe is -  (211, 3)


,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


c) If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. So for the 9th cell in the table on the Wikipedia page, the value of the Borough and the Neighborhood columns will be Queen's Park.

In [14]:
df_pcs['Neighborhood'] = np.where(df_pcs['Neighborhood'] == 'Not assigned', df_pcs['Borough'], df_pcs['Neighborhood'])
df_pcs.head(11)

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Queen's Park
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


In [15]:
print("Shape of Dataframe is - ",df_pcs.shape)

Shape of Dataframe is -  (211, 3)


d) More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma.

In [16]:
df_pcs_grouped = pd.DataFrame(df_pcs.groupby(['PostalCode','Borough'])['Neighborhood'].apply(','.join).reset_index())
df_pcs_grouped.head(11)

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


In [17]:
print("Shape of Dataframe with grouped Neiborhoods is - ",df_pcs_grouped.shape)

Shape of Dataframe with grouped Neiborhoods is -  (103, 3)


## Part 2

In order to utilize the Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood

a) Installing GeoCoder

In [9]:
!conda install -c conda-forge geocoder --yes
print("Installation Done!")
import geocoder # import geocoder
print("Library GeoCoder imported!")

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geocoder


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geocoder-1.38.1            |             py_1          53 KB  conda-forge
    openssl-1.1.1c             |       h516909a_0         2.1 MB  conda-forge
    certifi-2019.6.16          |           py36_1         149 KB  conda-forge
    ca-certificates-2019.6.16  |       hecc5488_0         145 KB  conda-forge
    ratelim-0.1.6              |             py_2           6 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geocoder:        1.38.1-py_1       conda-forge
    ratelim:         0.1.6-py_2        conda-forge

The following packages will be UPDATED:

    

In [10]:
# define geocoder function
def get_geocoder(postal_code_from_df):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code_from_df.strip()))
        lat_lng_coords = g.latlng
        latitude = lat_lng_coords[0]
        longitude = lat_lng_coords[1]
    return latitude,longitude

In [18]:
# Add columns
df_pcs_grouped['Latitude'], df_pcs_grouped['Longitude'] = zip(*df_pcs_grouped['PostalCode'].apply(get_geocoder))
df_pcs_grouped.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.811525,-79.195517
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.785730,-79.158750
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.765690,-79.175256
3,M1G,Scarborough,Woburn,43.768359,-79.217590
4,M1H,Scarborough,Cedarbrae,43.769688,-79.239440


In [20]:
print("Shape of Dataframe with grouped Neiborhoods is - ",df_pcs_grouped.shape)

Shape of Dataframe with grouped Neiborhoods is -  (103, 5)
